# 🤖 YouTube 자막 추출 + Gemini 요약 + Notion 연동 (통합)

**YouTube 영상 자막을 추출하고, Gemini AI로 요약한 후, Notion에 자동으로 저장하는 완전 자동화 시스템**

### 🎯 통합된 기능:
- ✅ YouTube 자막 추출 (최신 API 사용)
- ✅ Google Gemini Pro AI 요약
- ✅ Notion 페이지 자동 생성 및 저장
- ✅ 사용자 정의 요약 스타일 지원

In [ ]:
# 패키지 import
import asyncio
import json
import logging
import os
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Union
from urllib.parse import urlparse
import re

# 웹 크롤링 및 HTTP
import requests
from bs4 import BeautifulSoup
import validators

# Google GenAI
import google.generativeai as genai
from google.genai import types

# Notion API
from notion_client import Client
import numpy as np
import pandas as pd

# 유튜브 (최신 API)
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound

# 환경변수
from dotenv import load_dotenv
load_dotenv()

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ 모든 패키지가 성공적으로 로드되었습니다!")

## 🔑 API 키 및 환경 설정(설정완료로 패스)

In [ ]:
# 환경 변수 설정
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
PARENT_PAGE_ID = os.getenv("NOTION_PARENT_PAGE") # 상위 페이지 ID
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")  # Gemini API 키

print("🔑 환경 변수 상태:")
print(f"  • NOTION_TOKEN: {'✅ 설정됨' if NOTION_TOKEN else '❌ 설정 필요'}")
print(f"  • PARENT_PAGE_ID: {'✅ 설정됨' if PARENT_PAGE_ID else '❌ 설정 필요'}")
print(f"  • NOTION_DATABASE_ID: {'✅ 설정됨' if NOTION_DATABASE_ID else '❌ 설정 필요'}")
print(f"  • GEMINI_API_KEY: {'✅ 설정됨' if GEMINI_API_KEY else '❌ 설정 필요'}")

if not all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY]):
    print("⚠️ 필요한 환경 변수가 설정되지 않았습니다!")
    print("💡 .env 파일에 다음 항목들을 설정해주세요:")
    print("   - NOTION_TOKEN=your_notion_token")
    print("   - notion_parent_page=your_parent_page_id")
    print("   - GOOGLE_AI_API_KEY=your_google_ai_key")

BaseTool(모든 도구들의 기본 인터페이스)

In [ ]:
# Tool 기본 인터페이스
class BaseTool(ABC):
    """모든 도구의 기본 인터페이스"""
    
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
        self.logger = logging.getLogger(f"Tool.{name}")
    
    @abstractmethod
    async def execute(self, **kwargs) -> Dict[str, Any]:
        """도구 실행 (추상 메서드)"""
        pass
    
    def log_info(self, message: str):
        """로깅 헬퍼"""
        self.logger.info(f"[{self.name}] {message}")
        print(f"🔧 [{self.name}] {message}")
    
    def log_error(self, message: str):
        """에러 로깅 헬퍼"""
        self.logger.error(f"[{self.name}] {message}")
        print(f"❌ [{self.name}] {message}")

print("📐 BaseTool 인터페이스 정의 완료!")

YouTubeTranscriptTool(YouTube 자막 추출 도구 (다국어 지원))

In [ ]:
class YouTubeTranscriptTool(BaseTool):
    """YouTube 자막 추출 도구 (다국어 지원)"""
    
    def __init__(self):
        super().__init__("YouTubeTranscript", "YouTube 영상에서 자막을 추출하는 도구")
    
    def extract_video_id(self, url: str) -> str:
        """YouTube URL에서 비디오 ID 추출"""
        patterns = [
            r'(?:youtube\.com\/watch\?v=|youtu\.be\/)([^&\n?#]+)',
            r'youtube\.com\/embed\/([^&\n?#]+)',
            r'youtube\.com\/v\/([^&\n?#]+)',
            r'youtube\.com\/shorts\/([^&\n?#]+)'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                return match.group(1)
        return None
    
    async def execute(self, url: str, **kwargs) -> Dict[str, Any]:
        """YouTube 자막 추출 (최신 API 사용)"""
        try:
            self.log_info(f"자막 추출 시작: {url}")
            
            video_id = self.extract_video_id(url)
            if not video_id:
                return {
                    'success': False, 
                    'error': '유효하지 않은 YouTube URL입니다.',
                    'data': None
                }
            
            self.log_info(f"비디오 ID: {video_id}")
            
            # 최신 API 사용
            api = YouTubeTranscriptApi()
            
            # 다양한 언어로 시도
            languages_to_try = [['ko'], ['en'], ['a.ko'], ['a.en']]
            
            for languages in languages_to_try:
                try:
                    self.log_info(f"{languages[0]} 언어 자막 시도 중...")
                    fetched_transcript = api.fetch(video_id, languages=languages)
                    
                    # FetchedTranscriptSnippet 객체의 속성 접근
                    content = ' '.join([item.text for item in fetched_transcript])
                    content = re.sub(r'\s+', ' ', content).strip()
                    
                    self.log_info(f"자막 추출 성공! (언어: {languages[0]}, 길이: {len(content)}자)")
                    return {
                        'success': True,
                        'error': None,
                        'data': {
                            'content': content,
                            'language': languages[0],
                            'video_id': video_id,
                            'length': len(content),
                            'url': url
                        }
                    }
                except Exception as e:
                    self.log_info(f"{languages[0]} 실패: {str(e)[:50]}...")
                    continue
            
            # 사용 가능한 자막 목록 확인
            try:
                self.log_info("사용 가능한 자막 목록 확인 중...")
                transcript_list = api.list(video_id)
                
                if len(transcript_list) > 0:
                    first_transcript = transcript_list[0]
                    self.log_info(f"첫 번째 자막 언어로 시도: {first_transcript.language_code}")
                    
                    fetched_transcript = first_transcript.fetch()
                    content = ' '.join([item.text for item in fetched_transcript])
                    content = re.sub(r'\s+', ' ', content).strip()
                    
                    self.log_info(f"자막 추출 성공! (언어: {first_transcript.language_code}, 길이: {len(content)}자)")
                    return {
                        'success': True,
                        'error': None,
                        'data': {
                            'content': content,
                            'language': first_transcript.language_code,
                            'video_id': video_id,
                            'length': len(content),
                            'url': url
                        }
                    }
                
            except Exception as e:
                self.log_error(f"자막 목록 확인 실패: {str(e)}")
            
            return {
                'success': False, 
                'error': '자막을 찾을 수 없거나 네트워크 제한이 있습니다.',
                'data': None
            }
            
        except Exception as e:
            self.log_error(f"자막 추출 중 오류: {str(e)}")
            return {
                'success': False, 
                'error': f'오류 발생: {str(e)}',
                'data': None
            }

TranslationTool(Gemini AI 번역 도구)

In [ ]:
class TranslationTool(BaseTool):
    """Gemini AI 번역 도구"""
    
    def __init__(self, api_key: str, model_name: str = 'gemini-2.5-flash-lite'):
        super().__init__("Translation", "Gemini AI로 텍스트를 번역하는 도구")
        
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
        
        self.log_info(f"Gemini {model_name} 번역 모델 초기화 완료")
    
    async def execute(self, content: str, source_lang: str, target_lang: str = 'ko', 
                    **kwargs) -> Dict[str, Any]:
        """고품질 번역 실행"""
        try:
            self.log_info(f"번역 시작: {source_lang} → {target_lang} (길이: {len(content)}자)")
            
            # 번역 프롬프트
            translation_prompt = f"""
다음은 YouTube 영상의 {source_lang} 자막입니다. 이를 자연스럽고 정확한 {target_lang}로 번역해주세요.

## 번역 지침:
1. **의미 보존**: 원문의 의미를 정확히 전달
2. **자연스러운 번역**: 직역이 아닌 의역으로 자연스럽게
3. **전문용어 유지**: 기술용어나 고유명사는 적절히 보존
4. **맥락 고려**: 영상의 맥락을 고려하여 번역
5. **구어체 유지**: 영상의 톤앤매너 유지

번역할 내용:
{content}

위 내용을 {target_lang}로 번역해주세요. 번역된 내용만 출력하고, 다른 설명은 추가하지 마세요.
"""
            
            # Gemini 번역 요청
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.model.generate_content(
                    translation_prompt,
                    generation_config={
                        'temperature': 0.3,
                        'max_output_tokens': 2048,
                        'top_p': 0.8,
                        'top_k': 40
                    }
                )
            )
            
            if not response or not response.text:
                return {
                    'success': False,
                    'error': 'Gemini에서 번역을 생성하지 못했습니다.',
                    'data': None
                }
            
            translated_content = response.text.strip()
            
            self.log_info(f"번역 완료 (원본: {len(content)}자 → 번역: {len(translated_content)}자)")
            
            return {
                'success': True,
                'error': None,
                'data': {
                    'translated_content': translated_content,
                    'original_content': content,
                    'source_lang': source_lang,
                    'target_lang': target_lang,
                    'original_length': len(content),
                    'translated_length': len(translated_content)
                }
            }
            
        except Exception as e:
            self.log_error(f"번역 실패: {str(e)}")
            return {
                'success': False,
                'error': f'번역 실패: {str(e)}',
                'data': None
            }

print("🌍 TranslationTool 정의 완료!")

GeminiSummaryTool(Gemini AI 요약 도구)

In [ ]:
class GeminiSummaryTool(BaseTool):
    """Gemini AI 요약 도구"""
    
    def __init__(self, api_key: str, model_name: str = 'gemini-2.5-flash-lite'):
        super().__init__("GeminiSummary", "Gemini AI로 텍스트를 요약하는 도구")
        
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
        
        self.log_info(f"Gemini {model_name} 모델 초기화 완료")
    
    async def execute(self, content: str, custom_prompt: str = '', 
                    video_info: Dict = None, **kwargs) -> Dict[str, Any]:
        """요약 실행"""
        try:
            self.log_info(f"Gemini 요약 시작 (콘텐츠 길이: {len(content)}자)")
            
            # 기본 요약 프롬프트
            default_prompt = """
다음은 유튜브 영상의 자막입니다. 이를 학습자를 위해 체계적으로 요약해주세요.

## 📝 요약 구조
1. **핵심 주제와 목표** - 이 영상이 다루는 주요 내용
2. **주요 개념과 내용** - 핵심 포인트를 3-5개로 정리
3. **실습/예제 내용** - 코드나 실습이 있다면 상세히 기록
4. **학습 포인트와 결론** - 중요한 takeaway

각 섹션을 명확하게 구분하고, 마크다운 형식으로 구조화해서 작성해주세요.
"""
            
            system_prompt = custom_prompt if custom_prompt else default_prompt
            
            # 비디오 정보 추가
            video_context = ""
            if video_info:
                video_context = f"""
## 📺 영상 정보:
- 제목: {video_info.get('title', 'Unknown')}
- 채널: {video_info.get('channel', 'Unknown')}
- 비디오 ID: {video_info.get('video_id', 'Unknown')}
"""
            
            # 콘텐츠 길이 제한
            max_content_length = 25000
            if len(content) > max_content_length:
                content = content[:max_content_length] + "\n\n... (내용이 길어 일부만 요약됩니다)"
                self.log_info(f"콘텐츠가 길어서 {max_content_length}자로 축약")
            
            # 전체 프롬프트 구성
            full_prompt = f"{system_prompt}{video_context}\n\n## 원본 콘텐츠:\n{content}"
            
            # Gemini 호출
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.model.generate_content(
                    full_prompt,
                    generation_config={
                        'temperature': 0.3,
                        'max_output_tokens': 2048,
                        'top_p': 0.8,
                        'top_k': 40
                    }
                )
            )
            
            if not response or not response.text:
                return {
                    'success': False,
                    'error': 'Gemini에서 응답을 생성하지 못했습니다.',
                    'data': None
                }
            
            summary = response.text.strip()
            
            self.log_info(f"요약 생성 완료 (길이: {len(summary)}자)")
            
            return {
                'success': True,
                'error': None,
                'data': {
                    'summary': summary,
                    'original_length': len(content),
                    'summary_length': len(summary),
                    'tokens_used': len(full_prompt.split()) + len(summary.split())
                }
            }
            
        except Exception as e:
            self.log_error(f"요약 생성 실패: {str(e)}")
            return {
                'success': False,
                'error': f'Gemini 요약 실패: {str(e)}',
                'data': None
            }


YouTubeContentAgent (콘텐츠 요약 전문 에이전트)

In [ ]:
class ContentSummaryAgent(BaseAgent):
    """GeminiSummaryTool을 활용한 콘텐츠 요약 전문 에이전트"""
    
    def __init__(self, summary_tool: GeminiSummaryTool):
        super().__init__("ContentSummary", "GeminiSummaryTool을 활용한 콘텐츠 요약 전문 에이전트", [summary_tool])
        self.summary_tool = summary_tool
    
    async def execute(self, content: str, custom_prompt: str = '', 
                    video_info: Dict = None, **kwargs) -> Dict[str, Any]:
        """콘텐츠 요약 실행"""
        try:
            self.log_info(f"콘텐츠 요약 시작 (길이: {len(content)}자)")
            
            # Gemini 요약 도구 사용
            summary_result = await self.summary_tool.execute(
                content=content,
                custom_prompt=custom_prompt,
                video_info=video_info
            )
            
            return summary_result
            
        except Exception as e:
            self.log_error(f"콘텐츠 요약 실패: {str(e)}")
            return {
                'success': False,
                'error': f'콘텐츠 요약 실패: {str(e)}',
                'data': None
            }

print("📝 ContentSummaryAgent 정의 완료!")

GeminiSummaryTool(Gemini AI 요약 도구)

In [ ]:
class GeminiSummaryTool(BaseTool):
    """Gemini API를 호출하여 텍스트를 요약하는 도구"""
    
    def __init__(self, api_key: str, model_name: str = 'gemini-2.5-flash-lite'):
        super().__init__("GeminiSummary", "Gemini API를 호출하여 텍스트를 요약하는 도구")
        
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
        
        self.log_info(f"Gemini {model_name} 모델 초기화 완료")
    
    async def execute(self, content: str, custom_prompt: str = '', 
                    video_info: Dict = None, **kwargs) -> Dict[str, Any]:
        """요약 실행"""
        try:
            self.log_info(f"Gemini 요약 시작 (콘텐츠 길이: {len(content)}자)")
            
            # 기본 요약 프롬프트
            default_prompt = """
다음은 유튜브 영상의 자막입니다. 이를 학습자를 위해 체계적으로 요약해주세요.

## 📝 요약 구조
1. **핵심 주제와 목표** - 이 영상이 다루는 주요 내용
2. **주요 개념과 내용** - 핵심 포인트를 3-5개로 정리
3. **실습/예제 내용** - 코드나 실습이 있다면 상세히 기록
4. **학습 포인트와 결론** - 중요한 takeaway

각 섹션을 명확하게 구분하고, 마크다운 형식으로 구조화해서 작성해주세요.
"""
            
            system_prompt = custom_prompt if custom_prompt else default_prompt
            
            # 비디오 정보 추가
            video_context = ""
            if video_info:
                video_context = f"""
## 📺 영상 정보:
- 제목: {video_info.get('title', 'Unknown')}
- 채널: {video_info.get('channel', 'Unknown')}
- 비디오 ID: {video_info.get('video_id', 'Unknown')}
"""
            
            # 콘텐츠 길이 제한
            max_content_length = 25000
            if len(content) > max_content_length:
                content = content[:max_content_length] + "\n\n... (내용이 길어 일부만 요약됩니다)"
                self.log_info(f"콘텐츠가 길어서 {max_content_length}자로 축약")
            
            # 전체 프롬프트 구성
            full_prompt = f"{system_prompt}{video_context}\n\n## 원본 콘텐츠:\n{content}"
            
            # Gemini 호출
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.model.generate_content(
                    full_prompt,
                    generation_config={
                        'temperature': 0.3,
                        'max_output_tokens': 2048,
                        'top_p': 0.8,
                        'top_k': 40
                    }
                )
            )
            
            if not response or not response.text:
                return {
                    'success': False,
                    'error': 'Gemini에서 응답을 생성하지 못했습니다.',
                    'data': None
                }
            
            summary = response.text.strip()
            
            self.log_info(f"요약 생성 완료 (길이: {len(summary)}자)")
            
            return {
                'success': True,
                'error': None,
                'data': {
                    'summary': summary,
                    'original_length': len(content),
                    'summary_length': len(summary),
                    'tokens_used': len(full_prompt.split()) + len(summary.split())
                }
            }
            
        except Exception as e:
            self.log_error(f"요약 생성 실패: {str(e)}")
            return {
                'success': False,
                'error': f'Gemini 요약 실패: {str(e)}',
                'data': None
            }

ContentSummaryAgent(콘텐츠 요약 전문 에이전트)

In [ ]:
class ContentSummaryAgent(BaseAgent):
    """콘텐츠 요약 전문 에이전트"""
    
    def __init__(self, summary_tool: GeminiSummaryTool):
        super().__init__("ContentSummary", "텍스트 콘텐츠 요약 처리", [summary_tool])
        self.summary_tool = summary_tool
    
    async def execute(self, content: str, custom_prompt: str = '', 
                    video_info: Dict = None, **kwargs) -> Dict[str, Any]:
        """콘텐츠 요약 실행"""
        try:
            self.log_info(f"콘텐츠 요약 시작 (길이: {len(content)}자)")
            
            # Gemini 요약 도구 사용
            summary_result = await self.summary_tool.execute(
                content=content,
                custom_prompt=custom_prompt,
                video_info=video_info
            )
            
            return summary_result
            
        except Exception as e:
            self.log_error(f"콘텐츠 요약 실패: {str(e)}")
            return {
                'success': False,
                'error': f'콘텐츠 요약 실패: {str(e)}',
                'data': None
            }

print("📝 GeminiSummaryTool + ContentSummaryAgent 정의 완료!")

Notion Agent(페이지 생성 도구)

In [ ]:
class NotionPageTool(BaseTool):
    """Notion 페이지 생성 도구"""
    
    def __init__(self, notion_token: str, parent_page_id: str):
        super().__init__("NotionPage", "Notion에 페이지를 생성하는 도구")
        self.notion_token = notion_token
        self.parent_page_id = parent_page_id
        
        self.log_info("Notion API 클라이언트 초기화 완료")
    
    async def execute(self, title: str, summary: str, original_content: str, 
                    url: str, metadata: Dict = None, **kwargs) -> Dict[str, Any]:
        """Notion 페이지 생성 실행"""
        try:
            self.log_info(f"Notion 페이지 생성 시작: {title}")
            
            # Notion API 헤더
            headers = {
                "Authorization": f"Bearer {self.notion_token}",
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28"
            }
            
            # 현재 시간
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # 제목이 없으면 자동 생성
            if not title:
                title = f"YouTube 요약 - {current_time}"
            
            # 새 페이지 생성
            new_page = {
                "parent": {"page_id": self.parent_page_id},
                "properties": {
                    "title": [{"text": {"content": title}}]
                }
            }
            
            # 페이지 생성 요청
            page_response = requests.post(
                "https://api.notion.com/v1/pages", 
                headers=headers, 
                data=json.dumps(new_page)
            )
            
            if page_response.status_code != 200:
                return {
                    'success': False,
                    'error': f'페이지 생성 실패: {page_response.text}',
                    'data': None
                }
            
            page_data = page_response.json()
            page_id = page_data['id']
            page_url = page_data['url']
            
            self.log_info(f"페이지 생성 완료: {page_id}")
            
            # 콘텐츠 블록 생성
            blocks = self.create_content_blocks(summary, original_content, url, metadata)
            
            # 페이지에 블록 추가
            blocks_response = requests.patch(
                f"https://api.notion.com/v1/blocks/{page_id}/children",
                headers=headers,
                data=json.dumps({"children": blocks})
            )
            
            if blocks_response.status_code != 200:
                self.log_error(f"콘텐츠 추가 실패: {blocks_response.text}")
            else:
                self.log_info("페이지 콘텐츠 추가 완료")
            
            return {
                'success': True,
                'error': None,
                'data': {
                    'page_id': page_id,
                    'page_url': page_url,
                    'title': title
                }
            }
            
        except Exception as e:
            self.log_error(f"페이지 생성 실패: {str(e)}")
            return {
                'success': False,
                'error': f'Notion 페이지 생성 실패: {str(e)}',
                'data': None
            }
    
    def create_content_blocks(self, summary: str, original_content: str, 
                            url: str, metadata: Dict = None) -> List[Dict]:
        """Notion 페이지 콘텐츠 블록 생성 (기존 로직 그대로)"""
        blocks = []
        
        # 현재 시간
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # 1. YouTube URL 링크
        blocks.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {
                "rich_text": [
                    {"type": "text", "text": {"content": "🔗 원본 영상: "}},
                    {"type": "text", "text": {"content": url, "link": {"url": url}}}
                ]
            }
        })
        
        # 2. 메타데이터
        if metadata:
            meta_parts = []
            meta_parts.append(f"📊 언어: {metadata.get('language', 'unknown')}")
            meta_parts.append(f"길이: {metadata.get('length', 0):,}자")
            
            if metadata.get('translated'):
                meta_parts.append(f"번역: {metadata.get('original_language')} → 한국어")
            
            meta_parts.append(f"생성일: {current_time}")
            meta_text = " | ".join(meta_parts)
            
            blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text", "text": {"content": meta_text}}]
                }
            })
        
        # 3. 구분선
        blocks.append({"object": "block", "type": "divider", "divider": {}})
        
        # 4. AI 요약 제목
        blocks.append({
            "object": "block",
            "type": "heading_1",
            "heading_1": {
                "rich_text": [{"type": "text", "text": {"content": "🤖 Gemini AI 요약"}}]
            }
        })
        
        # 5. 요약 내용 (2000자씩 나누어 추가)
        summary_chunks = [summary[i:i+2000] for i in range(0, len(summary), 2000)]
        for chunk in summary_chunks:
            blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text", "text": {"content": chunk}}]
                }
            })
        
        # 6. 구분선
        blocks.append({"object": "block", "type": "divider", "divider": {}})
        
        # 7. 원문 자막 (토글 블록으로)
        original_chunks = [original_content[i:i+2000] for i in range(0, len(original_content), 2000)]
        original_blocks = []
        
        for chunk in original_chunks:
            original_blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text", "text": {"content": chunk}}]
                }
            })
        
        blocks.append({
            "object": "block",
            "type": "toggle",
            "toggle": {
                "rich_text": [{"type": "text", "text": {"content": "📄 원문 자막 보기"}}],
                "children": original_blocks[:50]  # 최대 50개 블록
            }
        })
        
        return blocks

NotionPublishAgent(발행 전문 에이전트)

In [ ]:
class NotionPublishAgent(BaseAgent):
    """Notion 발행 전문 에이전트"""
    
    def __init__(self, notion_tool: NotionPageTool):
        super().__init__("NotionPublish", "Notion에 콘텐츠 발행", [notion_tool])
        self.notion_tool = notion_tool
    
    async def execute(self, title: str, summary: str, original_content: str, 
                    url: str, metadata: Dict = None, **kwargs) -> Dict[str, Any]:
        """Notion 발행 실행"""
        try:
            self.log_info(f"Notion 페이지 발행: '{title}'")
            
            # Notion 페이지 생성 도구 사용
            notion_result = await self.notion_tool.execute(
                title=title,
                summary=summary,
                original_content=original_content,
                url=url,
                metadata=metadata
            )
            
            return notion_result
            
        except Exception as e:
            self.log_error(f"Notion 발행 실패: {str(e)}")
            return {
                'success': False,
                'error': f'Notion 발행 실패: {str(e)}',
                'data': None
            }

print("📄 NotionPageTool + NotionPublishAgent 정의 완료!")

YouTube-Gemini-Notion 워크플로우 코디네이터

In [ ]:
# ============================================================================
# YouTube-Gemini-Notion 워크플로우 코디네이터
# ============================================================================

class YouTubeGeminiNotionCoordinator:
    """YouTube → Gemini → Notion 전체 워크플로우 조율 코디네이터"""
    
    def __init__(self, 
                 youtube_agent: YouTubeContentAgent,
                 summary_agent: ContentSummaryAgent,
                 notion_agent: NotionPublishAgent):
        
        self.youtube_agent = youtube_agent
        self.summary_agent = summary_agent
        self.notion_agent = notion_agent
        
        self.logger = logging.getLogger("Coordinator.YouTubeGeminiNotion")
        
        print("🎯 YouTube-Gemini-Notion 코디네이터 초기화 완료!")
        print("📊 등록된 에이전트:")
        print(f"  • 🎬 YouTube 콘텐츠: {youtube_agent.name}")
        print(f"  • 📝 콘텐츠 요약: {summary_agent.name}")
        print(f"  • 📄 Notion 발행: {notion_agent.name}")
    
    def log_step(self, step: int, total: int, description: str):
        """단계별 로깅"""
        print(f"\n📍 [{step}/{total}] {description}")
        print("-" * 50)
    
    async def process_single_video(self, 
                                 url: str,
                                 custom_prompt: str = '',
                                 custom_title: str = '') -> Dict[str, Any]:
        """
        단일 비디오 처리 워크플로우 (기존 process_youtube_to_notion과 동일한 동작)
        
        Args:
            url: YouTube URL
            custom_prompt: 사용자 정의 요약 프롬프트
            custom_title: 사용자 정의 페이지 제목
            
        Returns:
            Dict: 전체 처리 결과
        """
        
        result = {
            'workflow': 'single_video',
            'url': url,
            'steps': {
                'youtube_content': {},
                'summary': {},
                'notion_publish': {}
            },
            'overall_success': False,
            'errors': [],
            'final_result': {}
        }
        
        try:
            print(f"\n" + "🚀 Agent 협업 워크플로우 시작!".center(80, "="))
            print(f"📺 URL: {url}")
            print("🌍 다국어 자막 지원 + 자동 번역 기능 활성화")
            
            # ========================================
            # 1단계: YouTube 콘텐츠 처리 (다국어 + 번역)
            # ========================================
            self.log_step(1, 3, "YouTube Content Agent - 자막 추출 + 자동 번역")
            
            youtube_result = await self.youtube_agent.execute(url=url)
            result['steps']['youtube_content'] = youtube_result
            
            if not youtube_result['success']:
                result['errors'].append(f"YouTube 에이전트 실패: {youtube_result['error']}")
                print(f"❌ 자막 추출 실패: {youtube_result['error']}")
                return result
            
            transcript_data = youtube_result['data']
            print(f"✅ YouTube 에이전트 성공 (언어: {transcript_data.get('language', 'unknown')}, 길이: {transcript_data.get('length', 0):,}자)")
            
            if transcript_data.get('translated'):
                print(f"🌍 자동 번역 완료: {transcript_data.get('original_language')} → 한국어")
            
            # ========================================
            # 2단계: AI 요약 생성
            # ========================================
            self.log_step(2, 3, "Content Summary Agent - Gemini AI 요약 생성")
            
            video_info = {
                'title': f"YouTube Video {transcript_data.get('video_id', 'Unknown')}",
                'channel': 'Unknown',
                'video_id': transcript_data.get('video_id', 'unknown')
            }
            
            summary_result = await self.summary_agent.execute(
                content=transcript_data['content'],
                custom_prompt=custom_prompt,
                video_info=video_info
            )
            result['steps']['summary'] = summary_result
            
            if not summary_result['success']:
                result['errors'].append(f"요약 에이전트 실패: {summary_result['error']}")
                print(f"❌ 요약 생성 실패: {summary_result['error']}")
                return result
            
            summary_data = summary_result['data']
            print(f"✅ Content Summary 에이전트 성공 (길이: {summary_data.get('summary_length', 0)}자, 토큰: {summary_data.get('tokens_used', 0):,})")
            
            # ========================================
            # 3단계: Notion 페이지 발행
            # ========================================
            self.log_step(3, 3, "Notion Publish Agent - 페이지 발행")
            
            # 제목 결정 (번역 정보 포함)
            if custom_title:
                final_title = custom_title
            else:
                final_title = f"YouTube 요약: {transcript_data.get('video_id', 'Unknown')}"
            
            # 번역 정보가 있으면 제목에 표시
            if transcript_data.get('translated'):
                orig_lang = transcript_data.get('original_language', 'unknown')
                final_title += f" [{orig_lang}→한국어 번역]"
            
            # 메타데이터 구성
            metadata = {
                'language': transcript_data.get('language', 'unknown'),
                'length': transcript_data.get('length', 0),
                'video_id': transcript_data.get('video_id', 'unknown'),
                'translated': transcript_data.get('translated', False),
                'original_language': transcript_data.get('original_language', 'unknown'),
                'tokens_used': summary_data.get('tokens_used', 0)
            }
            
            notion_result = await self.notion_agent.execute(
                title=final_title,
                summary=summary_data['summary'],
                original_content=transcript_data['content'],
                url=url,
                metadata=metadata
            )
            result['steps']['notion_publish'] = notion_result
            
            if not notion_result['success']:
                result['errors'].append(f"Notion 에이전트 실패: {notion_result['error']}")
                print(f"❌ Notion 페이지 생성 실패: {notion_result['error']}")
                return result
            
            notion_data = notion_result['data']
            print(f"✅ Notion Publish 에이전트 성공 (ID: {notion_data.get('page_id', 'unknown')})")
            
            # ========================================
            # 워크플로우 완료 (기존과 동일한 결과)
            # ========================================
            result['overall_success'] = True
            result['final_result'] = {
                'youtube_data': transcript_data,
                'summary_data': summary_data,
                'notion_data': notion_data,
                'page_url': notion_data.get('page_url'),
                'title': final_title
            }
            
            print(f"\n" + "🎉 Agent 협업 워크플로우 완료!".center(80, "="))
            print("✅ Tool → Agent → Coordinator 3계층 아키텍처 성공")
            print(f"📄 생성된 Notion 페이지: {notion_data.get('page_url', '')}")
            
            return result
            
        except Exception as e:
            error_msg = f"코디네이터 워크플로우 중 오류: {str(e)}"
            result['errors'].append(error_msg)
            print(f"❌ {error_msg}")
            return result

print("🎯 YouTubeGeminiNotionCoordinator 정의 완료!")

## 🚀 통합 Agent 초기화

In [ ]:
# ============================================================================
# 시스템 팩토리 - 전체 시스템을 쉽게 구성
# ============================================================================

class YouTubeGeminiNotionFactory:
    """YouTube-Gemini-Notion Agent 협업 시스템 팩토리"""
    
    @staticmethod
    def create_system(notion_token: str,
                     parent_page_id: str,
                     gemini_api_key: str,
                     gemini_model: str = 'gemini-2.5-flash-lite') -> YouTubeGeminiNotionCoordinator:
        """
        Agent 협업 시스템 생성
        
        Args:
            notion_token: Notion API 토큰
            parent_page_id: Notion 상위 페이지 ID
            gemini_api_key: Gemini API 키
            gemini_model: Gemini 모델명
            
        Returns:
            YouTubeGeminiNotionCoordinator: 설정된 코디네이터
        """
        
        print("🏗️ Agent 협업 시스템 구축 중...")
        print("📊 아키텍처: Tool → Agent → Coordinator 3계층")
        
        # ========================================
        # 1. Tool 계층 생성
        # ========================================
        print("\n🔧 Tool 계층 초기화 중...")
        
        # YouTube 도구들
        transcript_tool = YouTubeTranscriptTool()
        
        # AI 도구들 (번역 포함)
        summary_tool = GeminiSummaryTool(gemini_api_key, gemini_model)
        translation_tool = TranslationTool(gemini_api_key, gemini_model)
        
        # Notion 도구
        notion_tool = NotionPageTool(notion_token, parent_page_id)
        
        print(f"  ✅ YouTubeTranscriptTool: 다국어 자막 추출")
        print(f"  ✅ GeminiSummaryTool: AI 요약 ({gemini_model})")
        print(f"  ✅ TranslationTool: 자동 번역 ({gemini_model})")
        print(f"  ✅ NotionPageTool: 페이지 생성")
        # ========================================
        # 2. Agent 계층 생성
        # ========================================
        print("\n🤖 Agent 계층 초기화 중...")
        
        youtube_agent = YouTubeContentAgent(
            transcript_tool=transcript_tool,
            translation_tool=translation_tool  # 번역 기능 포함
        )
        
        summary_agent = ContentSummaryAgent(summary_tool)
        notion_agent = NotionPublishAgent(notion_tool)
        
        print(f"  ✅ YouTubeContentAgent: 자막 + 번역 처리")
        print(f"  ✅ ContentSummaryAgent: AI 요약 전문")
        print(f"  ✅ NotionPublishAgent: 페이지 발행 전문")
        # ========================================
        # 3. Coordinator 계층 생성
        # ========================================
        print("\n🎯 Coordinator 계층 초기화 중...")
        
        coordinator = YouTubeGeminiNotionCoordinator(
            youtube_agent=youtube_agent,
            summary_agent=summary_agent,
            notion_agent=notion_agent
        )
        
        print("\n🎉 Agent 협업 시스템 구축 완료!")
        print("📋 시스템 특징:")
        print("  • 🔧 모듈화: 독립적인 Tool들")
        print("  • 🤖 전문화: 각 영역별 Agent들")
        print("  • 🎯 조율: 체계적인 Coordinator")
        print("  • 🌍 다국어: 자동 번역 지원")
        print("  • 🛠️ 확장성: 새로운 Tool/Agent 쉽게 추가")
        
        return coordinator
    @staticmethod
    def from_env_vars() -> YouTubeGeminiNotionCoordinator:
        """환경 변수에서 Agent 협업 시스템 생성"""
        # 환경 변수 로드
        NOTION_TOKEN = os.getenv("NOTION_TOKEN")
        PARENT_PAGE_ID = os.getenv("NOTION_PARENT_PAGE") or os.getenv("notion_parent_page")
        GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_AI_API_KEY")
        
        # 필수 변수 확인
        required_vars = {
            'NOTION_TOKEN': NOTION_TOKEN,
            'PARENT_PAGE_ID': PARENT_PAGE_ID,
            'GEMINI_API_KEY': GEMINI_API_KEY
        }
        
        missing_vars = [name for name, value in required_vars.items() if not value]
        
        if missing_vars:
            raise ValueError(f"필수 환경 변수가 설정되지 않았습니다: {', '.join(missing_vars)}")
        
        print("🔑 환경 변수에서 Agent 협업 시스템 생성 중...")
        
        return YouTubeGeminiNotionFactory.create_system(
            notion_token=NOTION_TOKEN,
            parent_page_id=PARENT_PAGE_ID,
            gemini_api_key=GEMINI_API_KEY
        )

print("🏭 YouTubeGeminiNotionFactory 정의 완료!")

## 🎯 완전 자동화 실행

In [ ]:
# ========================================
# Agent 협업 시스템 초기화 및 실행
# ========================================

# 시스템 초기화
try:
    coordinator = YouTubeGeminiNotionFactory.from_env_vars()
    print("✅ Agent 협업 시스템 초기화 성공!")
    print("🌍 다국어 자막 지원 + 자동 번역 기능 활성화")
except Exception as e:
    print(f"❌ 시스템 초기화 실패: {str(e)}")
    print("💡 환경 변수를 확인해주세요:")
    print("   - NOTION_TOKEN")
    print("   - NOTION_PARENT_PAGE") 
    print("   - GEMINI_API_KEY")
    coordinator = None

# YouTube URL 입력 (기존과 동일)
if coordinator:
    print("\n🔗 요약하고 Notion에 저장할 YouTube URL을 입력하세요:")
    youtube_url = input("URL: ").strip()

    if not youtube_url:
        youtube_url = "https://youtu.be/6hSjOCUmjGQ?si=ES5FOYHaFjWtguUl"  # 기본 테스트 URL
        print(f"📺 기본 URL 사용: {youtube_url}")

    print(f"📺 처리할 URL: {youtube_url}")

    # 추가 옵션 입력 (선택사항)
    custom_prompt = input("\n💬 사용자 정의 요약 프롬프트 (엔터 시 기본값): ").strip()
    custom_title = input("📝 사용자 정의 제목 (엔터 시 자동 생성): ").strip()

    # Agent 협업 시스템 실행
    print(f"\n🚀 Agent 협업 워크플로우 시작...")
    result = await coordinator.process_single_video(
        url=youtube_url,
        custom_prompt=custom_prompt,
        custom_title=custom_title
    )

    # 결과 출력
    if result['overall_success']:
        print(f"\n🎉 모든 Agent 협업 완료!")
        final_result = result['final_result']
        
        print(f"📄 생성된 Notion 페이지: {final_result.get('page_url', '')}")
        
        # 다국어 처리 정보
        youtube_data = final_result['youtube_data']
        if youtube_data.get('translated'):
            print(f"🌍 자동 번역 완료: {youtube_data.get('original_language')} → 한국어")
        
        # 처리 통계
        summary_data = final_result['summary_data']
        print(f"📊 요약 길이: {summary_data.get('summary_length', 0)}자")
        print(f"⚡ 토큰 사용량: {summary_data.get('tokens_used', 0):,}")
        
    else:
        print(f"\n❌ 처리 실패:")
        for error in result['errors']:
            print(f"  • {error}")

else:
    print("❌ 시스템이 초기화되지 않아 실행할 수 없습니다.")

## 📋 완료!

### 🎉 YouTube + Gemini + Notion 통합 시스템 완성!

**🔄 완전한 자동화 파이프라인:**
1. ✅ **YouTube 자막 추출** - 최신 API로 다양한 언어 지원
2. ✅ **Gemini AI 요약** - Google의 최신 AI로 고품질 요약
3. ✅ **Notion 페이지 생성** - 구조화된 페이지로 자동 저장
4. ✅ **사용자 정의** - 프롬프트와 제목 커스터마이징 지원

### 💡 주요 특징:
- 🎯 **원클릭 자동화**: URL 입력만으로 전체 프로세스 완료
- 📊 **풍부한 메타데이터**: 언어, 길이, 토큰 사용량 등 상세 정보
- 📝 **구조화된 페이지**: 요약 + 원문 자막을 깔끔하게 정리
- 🔗 **편리한 접근**: 생성된 Notion 페이지 직접 링크 제공

### 🛠️ 환경 설정 요구사항:
```env
NOTION_TOKEN=" "
notion_parent_page=" "
GEMINI_API_KEY=" "
NOTION_DATABASE_ID=" "
```

### 🔗 관련 파일:
- **[youtube_gemini_summary.py](computer:///mnt/user-data/outputs/youtube_gemini_summary.py)** - 단독 스크립트 버전
- **[ai_agent_backend.py](computer:///mnt/user-data/outputs/ai_agent_backend.py)** - 전체 백엔드 시스템

**이제 YouTube 영상을 입력하면 자동으로 요약되어 Notion에 깔끔하게 정리됩니다!** 🚀✨